In [1]:
from transformers import Tool, load_tool, ReactCodeAgent, HfEngine
import requests
import time

class WeatherTool(Tool):
    name = "weather_api"
    description = "Retrieves weather information for a specific city."
    inputs = {
        "city": {
            "type": "string",
            "description": "The name of the city to get weather information for."
        },
        "forecast_type": {
            "type": "string",
            "description": "Either 'current' for current weather or 'forecast' for future weather."
        }
    }
    output_type = "text"

    def __init__(self, api_key):
        super().__init__()
        self.api_key = api_key

    def __call__(self, city: str, forecast_type: str = "current") -> str:
        base_url = "http://api.openweathermap.org/data/2.5"
        
        if forecast_type == "current":
            endpoint = f"{base_url}/weather"
        else:
            endpoint = f"{base_url}/forecast"
        
        params = {
            "q": city,
            "appid": self.api_key,
            "units": "metric"
        }
        
        response = requests.get(endpoint, params=params)
        
        if response.status_code == 200:
            data = response.json()
            if forecast_type == "current":
                temp = data['main']['temp']
                description = data['weather'][0]['description']
                return f"The current weather in {city} is {description} with a temperature of {temp}°C."
            else:
                forecast = data['list'][0]  # Get the first forecast entry
                temp = forecast['main']['temp']
                description = forecast['weather'][0]['description']
                date_time = forecast['dt_txt']
                return f"The forecast for {city} on {date_time} is {description} with a temperature of {temp}°C."
        else:
            return f"Error: Unable to fetch weather data for {city}. Status code: {response.status_code}"

# Initialize the WeatherTool
api_key = 'c6dfc4d92a8f972d237ef696ec87b37a'
weather_tool = WeatherTool(api_key)

# Initialize the LLM engine with retry logic
def initialize_llm_engine(model_name, max_retries=5, wait_time=30):
    retries = 0
    while retries < max_retries:
        try:
            llm_engine = HfEngine(model_name)
            return llm_engine
        except Exception as e:
            print(f"Error initializing LLM engine: {e}")
            print(f"Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
            retries += 1
    raise Exception("Failed to initialize LLM engine after multiple attempts.")

llm_engine = initialize_llm_engine("../models/unsloth/Phi-3-mini-4k-instruct/")

# Initialize the agent with the weather tool
agent = ReactCodeAgent(tools=[weather_tool], llm_engine=llm_engine)

# Run it!
result = agent.run(
    "What's the weather like in London today?",
)
print(result)


/home/vatsal-patel/anaconda3/envs/unsloth_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
======== New task ========
What's the weather like in London today?

KeyboardInterrupt



In [3]:
# Initialize the WeatherTool
api_key = 'c6dfc4d92a8f972d237ef696ec87b37a'
weather_tool = WeatherTool(api_key)

# Test the WeatherTool independently
print(weather_tool("Kalyani", "current"))
print(weather_tool("London", "forecast"))


The current weather in Kalyani is overcast clouds with a temperature of 28.95°C.
The forecast for London on 2024-06-27 21:00:00 is broken clouds with a temperature of 18.46°C.


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load the model and tokenizer locally
model_name = "../models/unsloth/Phi-3-mini-4k-instruct/"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create a pipeline for text generation
text_gen_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Test the pipeline
# prompt = "What's the weather like in London today?"
prompt = "Tell me a joke?"
result = text_gen_pipeline(prompt, max_length=100)
print(result)


Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:07<00:00,  3.84s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.

KeyboardInterrupt



In [6]:
!ls

1_Data-Generation-OpenWeather-API-Copy1.ipynb	data
1_Data-Generation-OpenWeather-API.ipynb		llama.cpp
2_Model_Train_Phi3_Mini_4k_iunsloth_Copy.ipynb	logs
2_Model-Train-Phi3-Mini-4k-iunsloth.ipynb	outputs_combined
3_Inference_Pipeline_Setup.ipynb		_unsloth_sentencepiece_temp
4_Inference_with_llama_cpp.ipynb		Untitled1.ipynb
5_ZeroShot_inference.ipynb
